In [2]:
# download the dataset and save it in the same dir with this file.
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import KFold
from sklearn.pipeline import Pipeline
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.gaussian_process import GaussianProcessRegressor
from sklearn.gaussian_process.kernels import DotProduct, WhiteKernel,RationalQuadratic,Exponentiation
from sklearn.tree import DecisionTreeRegressor
from sklearn.kernel_ridge import KernelRidge
from sklearn.neural_network import MLPRegressor
from sklearn.preprocessing import PolynomialFeatures
from sklearn.model_selection import train_test_split
from sklearn import neighbors
import os
kf = KFold(n_splits=10)
# import scikit learn
from sklearn import linear_model,svm
from sklearn.preprocessing import PolynomialFeatures

def getxandy(filename):
  with open(filename,'r') as f:
    df = pd.read_csv(f,index_col=0)
    x = np.zeros((len(df.x),2))
    y = np.zeros((len(df.x)))
    x[:,0] = df.x.to_numpy()
    x[:,1] = df.y.to_numpy()
    y = df.z.to_numpy()
  return x,y

def getpolyxandy(filename,poly=2,withy=True):
  with open(filename,'r') as f:
    df = pd.read_csv(f,index_col=0)
    x = np.zeros((len(df.x),2))
    y = np.zeros((len(df.x)))
    x[:,0] = df.x.to_numpy()
    x[:,1] = df.y.to_numpy()
    if withy:
      y = df.z.to_numpy()
    else:
      y = None
    poly = PolynomialFeatures(degree=poly)
    x = poly.fit_transform(x)
  return x,y

def scoreperdataset(y_pred,y_test):
  return np.sqrt(np.sum(np.square(y_pred - y_test))/len(y_pred))

def writeresults(X_test,y_test,y_pred,f):
    idx = np.argsort(X_test[:,0])
    X_test = X_test[idx]
    y_test = y_test[idx]
    y_pred = y_pred[idx]
    data = {}
    data['x'] = X_test[:,0]
    data['y'] = X_test[:,1]
    data['z_test'] = y_test[idx]
    data['z_pred'] = y_pred[idx]
    df = pd.DataFrame(data)
    if not os.path.isdir('results/'):
      os.mkdir('results')
    df.to_csv('results/'+f+'.csv',float_format="%.6f")

In [25]:
# post 9
finalacc_small = []
trainfolder = 'small_datasets'
datasets = [chr(ord('A') + x) for x in range(18)]
for d in datasets:
  X,y = getpolyxandy(trainfolder+'/'+ d +'/train.csv',4)
  #X,y = getxandy(trainfolder+'/'+d+'/train.csv')
  reg = neighbors.KNeighborsRegressor(40, weights='distance')
  X_train, X_test, y_train, y_test = train_test_split(X,y,test_size=0.1)
  reg.fit(X_train,y_train)
  y_pred = reg.predict(X_test)
  loss_small = scoreperdataset(y_pred, y_test)
  finalacc_small.append(loss_small)
  print("{} loss {:.5f} ".format(d, loss_small))
  #break
  #writeresults(X_test,y_test,y_pred,f[9:])
print('final avg loss is {:.5f}'.format(sum(finalacc_small)/len(finalacc_small)))

A loss 0.10580 
B loss 0.15585 
C loss 0.37712 
D loss 0.02422 
E loss 0.03995 
F loss 0.20105 
G loss 0.01413 
H loss 0.10852 
I loss 0.17329 
J loss 0.54361 
K loss 0.54745 
L loss 0.65704 
M loss 0.53190 
N loss 0.53228 
O loss 0.56777 
P loss 0.52624 
Q loss 0.54025 
R loss 0.56787 
final avg loss is 0.34524


In [34]:
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import train_test_split
final_acc = []
pred_array = {}
trainfolder = 'small_datasets/'
datasets = [chr(ord('A') + x) for x in range(18)]
polydegree=4
for d in datasets:
  X,y = getpolyxandy(trainfolder+d+'/train.csv',polydegree)
  clf = GridSearchCV(neighbors.KNeighborsRegressor(weights='distance'), {'n_neighbors':[x for x in range(1,150)]},n_jobs=30)
  X_train,X_test,y_train,y_test = train_test_split(X,y,test_size=0.1, random_state=1)
  clf.fit(X_train, y_train)
  y_pred = clf.best_estimator_.predict(X_test)
  loss = scoreperdataset(y_pred,y_test)
  final_acc.append(loss)
  X_real,y_real = getpolyxandy(trainfolder+d+'/test.csv',polydegree,withy=False)
  y_real_pred = clf.best_estimator_.predict(X_real)
  pred_array[d+'.z'] = y_real_pred
  print('loss for {} is {}, maxdepth is {}'.format(d,loss, clf.best_estimator_.n_neighbors))
print('final avg loss is {:.5f}'.format( sum(final_acc)/len(final_acc)))
outdf = pd.DataFrame(pred_array)
outdf.to_csv('submission_knn_small.csv',index_label='id')

loss for A is 0.06412215354559062, maxdepth is 6
loss for B is 0.0860291950158973, maxdepth is 5
loss for C is 0.27330082603780265, maxdepth is 4
loss for D is 0.01950141390305742, maxdepth is 10
loss for E is 0.026632842522355468, maxdepth is 7
loss for F is 0.14575788781934573, maxdepth is 4
loss for G is 0.010822928064590301, maxdepth is 8
loss for H is 0.08082108592907368, maxdepth is 4
loss for I is 0.12604355286503405, maxdepth is 4
loss for J is 0.545831515091561, maxdepth is 33
loss for K is 0.5421207100548354, maxdepth is 22
loss for L is 0.6201324636392044, maxdepth is 11
loss for M is 0.5269188279644609, maxdepth is 117
loss for N is 0.5285231704701214, maxdepth is 86
loss for O is 0.5693892956537249, maxdepth is 26
loss for P is 0.5229880045531884, maxdepth is 149
loss for Q is 0.5503204295836411, maxdepth is 51
loss for R is 0.36882017886322005, maxdepth is 6
final avg loss is 0.31156


In [32]:
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import train_test_split
final_acc = []
pred_array = {}
trainfolder = 'small_datasets/'
datasets = [chr(ord('A') + x) for x in range(18)]
polydegree=4
for d in datasets:
  X,y = getpolyxandy(trainfolder+d+'/train.csv',polydegree)
  clf = GridSearchCV(neighbors.KNeighborsRegressor(weights='distance'), {'n_neighbors':[x for x in range(1,150)]},n_jobs=30)
  X_train,X_test,y_train,y_test = train_test_split(X,y,test_size=0.1, random_state=1)
  clf.fit(X_train, y_train)
  y_pred = clf.best_estimator_.predict(X_test)
  loss = scoreperdataset(y_pred,y_test)
  final_acc.append(loss)
  X_real,y_real = getpolyxandy(trainfolder+d+'/test.csv',polydegree,withy=False)
  y_real_pred = clf.best_estimator_.predict(X_real)
  pred_array[d+'.z'] = y_real_pred
  print('loss for {} is {}, maxdepth is {}'.format(d,loss, clf.best_estimator_.n_neighbors))
print('final avg loss is {:.5f}'.format( sum(final_acc)/len(final_acc)))
outdf = pd.DataFrame(pred_array)
outdf.to_csv('submission_knn_small.csv',index_label='id')

In [20]:
# pre 9
trainfolder = 'small_datasets'
datasets = [chr(ord('A') + x) for x in range(18)]
for d in datasets[:9]:
  X,y = getpolyxandy(trainfolder+'/'+ d +'/train.csv',4)
  #X,y = getxandy(trainfolder+'/'+d+'/train.csv')
  reg = DecisionTreeRegressor(max_depth=25)
  X_train, X_test, y_train, y_test = train_test_split(X,y,test_size=0.1)
  reg.fit(X_train,y_train)
  y_pred = reg.predict(X_test)
  loss_small = scoreperdataset(y_pred, y_test)
  finalacc_small.append(loss_small)
  print("{} loss {:.5f} ".format(d, loss_small))
  #break
  #writeresults(X_test,y_test,y_pred,f[9:])
print('final avg loss is {:.5f}'.format(sum(finalacc_small)/len(finalacc_small)))

A loss 0.13268 
B loss 0.16972 
C loss 0.41928 
D loss 0.03774 
E loss 0.05400 
F loss 0.22874 
G loss 0.01870 
H loss 0.12534 
I loss 0.21146 
final avg loss is 0.36533


In [47]:
# pre 9 large

#model = Pipeline([('poly', PolynomialFeatures(degree=4)),('linear', DecisionTreeRegressor(max_depth=25))])

finalacc_large = []
trainfolder = 'large_datasets'
datasets = [chr(ord('A') + x) for x in range(18)]
for d in datasets:
  X,y = getpolyxandy(trainfolder+'/'+ d +'/train.csv',4)
  reg = DecisionTreeRegressor(max_depth=25)
  X_train, X_test, y_train, y_test = train_test_split(X,y,test_size=0.1)
  reg.fit(X_train,y_train)
  y_pred = reg.predict(X_test)
  loss_large = scoreperdataset(y_pred, y_test)
  finalacc_large.append(loss)
  print("{} loss {:.5f}".format(d, loss_large))
  #writeresults(X_test,y_test,y_pred)
print('final average loss is {:.5f}'.format( sum(finalacc_large)/len(finalacc_large) ) )

KeyboardInterrupt: 

J loss 0.74672 
K loss 0.74609 
L loss 0.86362 
M loss 0.72611 
N loss 0.72007 
O loss 0.76649 
P loss 0.71774 
Q loss 0.74884 
R loss 0.60390 
final avg loss is 0.73773


In [58]:
# pre 9
finalacc_small = []
trainfolder = 'small_datasets'
datasets = [chr(ord('A') + x) for x in range(18)]
for d in datasets[9:]:
  X,y = getpolyxandy(trainfolder+'/'+ d +'/train.csv',4)
  reg = DecisionTreeRegressor(max_depth=25)
  X_train, X_test, y_train, y_test = train_test_split(X,y,test_size=0.1)
  reg.fit(X_train,y_train)
  y_pred = reg.predict(X_test)
  loss_small = scoreperdataset(y_pred, y_test)
  finalacc_small.append(loss_small)
  print("{} loss {:.5f} ".format(d, loss_small))
  #writeresults(X_test,y_test,y_pred,f[9:])
print('final avg loss is {:.5f}'.format(sum(finalacc_small)/len(finalacc_small)))

J loss 0.74384 
K loss 0.74708 
L loss 0.84819 
M loss 0.73457 
N loss 0.72024 
O loss 0.76775 
P loss 0.70474 
Q loss 0.73537 
R loss 0.58388 
final avg loss is 0.73174


In [20]:
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import train_test_split
final_acc = []
pred_array = {}
trainfolder = 'small_datasets/'
datasets = [chr(ord('A') + x) for x in range(18)]
polydegree=4
for d in datasets:
  X,y = getpolyxandy(trainfolder+d+'/train.csv',polydegree)
  clf = GridSearchCV(DecisionTreeRegressor(), {'max_depth':[10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30]},n_jobs=40)
  X_train,X_test,y_train,y_test = train_test_split(X,y,test_size=0.1, random_state=1)
  clf.fit(X_train, y_train)
  y_pred = clf.best_estimator_.predict(X_test)
  loss = scoreperdataset(y_pred,y_test)
  final_acc.append(loss)
  X_real,y_real = getpolyxandy(trainfolder+d+'/test.csv',polydegree,withy=False)
  y_real_pred = clf.best_estimator_.predict(X_real)
  pred_array[d+'.z'] = y_real_pred
  print('loss for {} is {}, maxdepth is {}'.format(d,loss, clf.best_estimator_.max_depth))
print('final avg loss is {:.5f}'.format( sum(final_acc)/len(final_acc)))
outdf = pd.DataFrame(pred_array)
outdf.to_csv('submission_{}_pd{}.csv'.format(trainfolder,polydegree),index_label='id')

loss for A is 0.12978216995160374, maxdepth is 29
loss for B is 0.17078259486046513, maxdepth is 30
loss for C is 0.430947655937802, maxdepth is 24
loss for D is 0.03718906086244351, maxdepth is 28
loss for E is 0.05243262217126007, maxdepth is 28
loss for F is 0.22230850240283162, maxdepth is 24
loss for G is 0.019695002501374292, maxdepth is 29
loss for H is 0.12333127560912857, maxdepth is 24
loss for I is 0.20551249823222553, maxdepth is 29
loss for J is 0.6631680250078472, maxdepth is 15
loss for K is 0.6723312829468607, maxdepth is 16
loss for L is 0.7761177173452065, maxdepth is 16
loss for M is 0.562200276208805, maxdepth is 11
loss for N is 0.5794837149891852, maxdepth is 12
loss for O is 0.6672138611018428, maxdepth is 13
loss for P is 0.5486853983740958, maxdepth is 10
loss for Q is 0.6013389282433018, maxdepth is 12
loss for R is 0.5300511255323415, maxdepth is 23
final avg loss is 0.38848


In [13]:
outdf

,A.z,B.z,C.z,D.z,E.z,F.z,G.z,H.z,I.z,J.z,K.z,L.z,M.z,N.z,O.z,P.z,Q.z,R.z
0,0.068590,-0.423999,-1.939186,0.276854,0.178814,-0.977992,-0.293425,-0.966794,0.405307,-1.535750,-1.621131,-0.939400,-0.870400,-1.226403,-0.953048,-0.916811,-1.092569,0.385946
1,0.578736,-0.083078,-0.997874,0.344362,0.156215,-0.774018,-0.255365,-0.726360,0.439748,-0.878575,-0.399362,-1.579657,-0.826202,-0.818445,-0.354231,-0.752633,-1.451437,0.422646
2,-1.385957,-1.820379,-3.232417,-0.984805,-1.474663,-3.052878,-0.462948,-2.074112,-2.820309,-2.681391,-2.759769,-4.749379,-3.099284,-3.331835,-3.841978,-2.692213,-2.756777,-2.174585
3,-0.269962,0.086945,0.610890,-0.846705,-0.613285,-0.467018,-0.366934,-0.578218,-0.011630,0.154351,0.150065,0.532862,-1.126181,-0.890864,-0.653721,-0.760812,-0.737915,0.122218
4,-0.138184,-0.170073,0.358636,-0.637182,-0.659206,0.112100,-0.409993,0.381512,0.453297,0.094388,-0.336522,-0.815501,1.215569,0.576633,0.432110,2.443393,1.695733,0.383195
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9995,-1.544600,-1.653927,-0.903935,-1.147495,-1.343468,-0.905728,-1.688055,-1.180908,-1.042003,-0.648542,-1.007877,-0.250274,-0.226460,-0.562846,-0.834183,-0.759265,-0.854740,-0.833198
9996,0.716113,0.205424,-0.095451,-0.660184,-0.719738,-1.136995,-0.185367,-1.134630,-0.652328,-0.413838,-0.428255,0.144051,-2.634563,-2.429890,-1.997505,-2.070956,-1.812553,-0.505799
9997,-1.146646,-0.749684,0.618468,-0.976357,-1.446850,-1.532404,-0.218184,-1.043520,-1.310997,-1.468794,-1.094033,-0.669335,-3.099284,-2.511635,-1.513668,-2.060561,-1.814404,-1.203519
9998,-0.860182,-0.001287,1.070914,0.843645,1.056504,2.046295,-0.360000,1.816746,3.581599,1.601896,0.997566,1.511469,2.660580,2.186360,2.651736,1.408421,2.149703,4.181706


In [22]:
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import train_test_split
final_acc = []
pred_array = {}
trainfolder = 'large_datasets/'
datasets = [chr(ord('A') + x) for x in range(18)]
polydegree=4
for d in datasets:
  X,y = getpolyxandy(trainfolder+d+'/train.csv',polydegree)
  clf = GridSearchCV(DecisionTreeRegressor(), {'max_depth':[x for x in range(10,40)]},n_jobs=20)
  X_train,X_test,y_train,y_test = train_test_split(X,y,test_size=0.1, random_state=1)
  clf.fit(X_train, y_train)
  y_pred = clf.best_estimator_.predict(X_test)
  loss = scoreperdataset(y_pred,y_test)
  final_acc.append(loss)
  X_real,y_real = getpolyxandy(trainfolder+d+'/test.csv',polydegree,withy=False)
  y_real_pred = clf.best_estimator_.predict(X_real)
  pred_array[d+'.z'] = y_real_pred
  print('loss for {} is {}, maxdepth is {}'.format(d,loss, clf.best_estimator_.max_depth))
print('final avg loss is {:.5f}'.format( sum(final_acc)/len(final_acc)))
outdf = pd.DataFrame(pred_array)
outdf.to_csv('submission_{}_pd{}.csv'.format(trainfolder,polydegree),index_label='id')

loss for A is 0.038441928844226855, maxdepth is 30
loss for B is 0.05409639127831196, maxdepth is 30


WorkerInterrupt: 